# Service ID analysis

In [1]:
from custom_functions.gtfs_methods import headways, convert_jsons_to_df, assess_qos_metrics,extract_schedules_from_observed, get_largest_group
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
try:
    #headway_t = pd.read_pickle("data/computed/theoretical_schedule.pkl")
    headway_t_qos = pd.read_pickle("data/computed/theoretical_schedule_qos.pkl")
except Exception as e:
    raise e

calendar_dates = pd.read_csv('data/timetables/gtfs3Sept/calendar_dates.txt', sep = ",")
calendar = pd.read_pickle("data/computed/gtfs3_/calendar.pkl")
trips = pd.read_csv('data/timetables/gtfs3Sept/trips.txt', sep = ",")
stops = pd.read_csv('data/timetables/gtfs3Sept/stops.txt', sep = ",")
stop_times = pd.read_csv('data/timetables/gtfs3Sept/stop_times.txt', sep = ",")
routes = pd.read_csv('data/timetables/gtfs3Sept/routes.txt', sep = ",")

In [9]:
calendar[calendar.service_id == 236270000]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label
3,236270000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays


In [5]:
calendar[calendar.label == "workdays"]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label
3,236270000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
15,237645001,1,1,1,1,1,0,0,20210906,20210917,2021-09-06,2021-09-17,workdays
26,237648001,1,1,1,1,1,0,0,20210906,20210917,2021-09-06,2021-09-17,workdays
34,237534000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
43,236695000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,237121001,1,1,1,1,1,0,0,20210903,20210917,2021-09-03,2021-09-17,workdays
418,237241001,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
421,237242001,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
423,237244001,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays


## Droping out of validity service IDs

Looking for services that finished before our data start and starts after.

In [7]:
obs = pd.read_pickle("data/computed/observed_schedule_dist_0.pkl")

In [8]:
min_date = pd.to_datetime(min(obs["date"]), format='%d-%m-%Y')
max_date = pd.to_datetime(max(obs["date"]), format='%d-%m-%Y')
min_date, max_date

(Timestamp('2021-09-06 00:00:00'), Timestamp('2021-09-21 00:00:00'))

In [6]:
calendar["start_date_ft"] = pd.to_datetime(calendar.start_date, format='%Y%m%d')
calendar["end_date_ft"] = pd.to_datetime(calendar.end_date, format='%Y%m%d')

Those that starts where our data ends

In [7]:
calendar[(calendar.start_date_ft > max_date)]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft


Those that ends before our data starts :

In [8]:
calendar[(calendar.end_date_ft < min_date)]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft


Mixing both conditions :

In [9]:
calendar[-((calendar.start_date_ft > max_date) | (calendar.end_date_ft < min_date))].to_pickle("data/computed/gtfs3_/calendar.pkl")

## Labelling service_ids

In [13]:
calendar[(calendar.monday == 1) & (calendar.tuesday == 1) & (calendar.wednesday == 1) & (calendar.thursday == 1) & (calendar.friday == 1)]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft
3,236270000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17
15,237645001,1,1,1,1,1,0,0,20210906,20210917,2021-09-06,2021-09-17
26,237648001,1,1,1,1,1,0,0,20210906,20210917,2021-09-06,2021-09-17
34,237534000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17
43,236695000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17
...,...,...,...,...,...,...,...,...,...,...,...,...
388,237121001,1,1,1,1,1,0,0,20210903,20210917,2021-09-03,2021-09-17
418,237241001,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17
421,237242001,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17
423,237244001,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17


In [7]:
rec = []
for name, row in calendar.iterrows():
    d = {
        0 : "monday",
        1 : "tuesday",
        2 : "wednesday",
        3 : "thursday",
        4 : "friday",
        5 : "saturday",
        6 : "sunday"
    }
    st = str(row.monday) + str(row.tuesday) + str(row.wednesday) + str(row.thursday) + str(row.friday) + str(row.saturday) + str(row.sunday)
    if st == "1111100": 
        val = "workdays"
    elif st == "0000011" :
        val = "week-end"
    else:
        val = ""
        for index, value in enumerate(st):
            if value == '1':
                if len(val) == 0:
                    val += d.get(index)
                elif len(val) != 0:
                    val += "-" + d.get(index)
    rec.append(val)

calendar["label"] = rec
calendar.to_pickle("data/computed/gtfs3_/calendar.pkl")

## Exceptions

In [9]:
obs = pd.read_pickle("data/computed/observed_schedule_dist_0.pkl")
min_date = pd.to_datetime(min(obs["date"]), format='%d-%m-%Y')
max_date = pd.to_datetime(max(obs["date"]), format='%d-%m-%Y')
min_date, max_date

(Timestamp('2021-09-06 00:00:00'), Timestamp('2021-09-21 00:00:00'))

In [10]:
calendar_dates["date_ft"] = pd.to_datetime(calendar_dates["date"], format='%Y%m%d')
calendar_dates

,service_id,date,exception_type,date_ft
0,237476501,20210912,1,2021-09-12
1,237566502,20210919,1,2021-09-19
2,238162502,20210919,1,2021-09-19
3,238164502,20210919,1,2021-09-19
4,238234501,20210919,1,2021-09-19
...,...,...,...,...
306,235954071,20210827,2,2021-08-27
307,235005071,20210827,2,2021-08-27
308,237243501,20210919,1,2021-09-19
309,235536070,20210827,2,2021-08-27


In [19]:
c = calendar_dates[(calendar_dates.date_ft >= min_date) & (calendar_dates.date_ft <= max_date)]

In [25]:
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label
3,236270000,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays
4,236269500,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday
6,237476501,0,0,0,0,0,1,0,20210912,20210912,2021-09-12,2021-09-12,saturday
7,237566502,0,0,0,0,0,1,0,20210919,20210919,2021-09-19,2021-09-19,saturday
15,237645001,1,1,1,1,1,0,0,20210906,20210917,2021-09-06,2021-09-17,workdays
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,237104001,1,1,1,1,1,0,0,20210906,20210917,2021-09-06,2021-09-17,workdays
432,237102501,0,0,0,0,0,1,0,20210911,20210911,2021-09-11,2021-09-11,saturday
433,237472602,0,0,0,0,0,0,1,20210912,20210912,2021-09-12,2021-09-12,sunday
435,238036502,0,0,0,0,0,1,0,20210918,20210918,2021-09-18,2021-09-18,saturday


In [28]:
c["day"] = c.date_ft.dt.strftime("%A")
c["day_l"] = np.where(
    c.day == "Sunday",
    "sunday",
    np.where(
        c.day == "Saturday",
        "saturday",
        "workdays"
    )
)
c_bis = c.merge(
    calendar,
    "left",
    "service_id"
)[["service_id","exception_type", "date_ft","day_l", "label"]]
c_bis

/var/folders/dz/p7gp3f295wxf90tn1dv3qkq40000gn/T/ipykernel_6495/28775010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c["day"] = c.date_ft.dt.strftime("%A")
/var/folders/dz/p7gp3f295wxf90tn1dv3qkq40000gn/T/ipykernel_6495/28775010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c["day_l"] = np.where(


,service_id,exception_type,date_ft,day_l,label
0,237476501,1,2021-09-12,sunday,saturday
1,237566502,1,2021-09-19,sunday,saturday
2,238162502,1,2021-09-19,sunday,saturday
3,238164502,1,2021-09-19,sunday,saturday
4,238234501,1,2021-09-19,sunday,saturday
...,...,...,...,...,...
271,237121001,2,2021-09-14,workdays,workdays
272,237121001,2,2021-09-15,workdays,workdays
273,237121001,2,2021-09-16,workdays,workdays
274,237243501,1,2021-09-19,sunday,saturday
